## Test R3-RAG-CS-Qwen

In [ ]:
# Vérifier les ressources disponibles
!nvidia-smi
!df -h
!free -h

# Installation des packages requis
!pip install torch transformers accelerate
!pip install vllm>=0.2.0
!pip install datasets
!pip install faiss-cpu  # Pour la recherche vectorielle
!pip install sentence-transformers

In [ ]:
import os
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Cloner le repository
!git clone https://github.com/Yuan-Li-FNLP/R3-RAG.git
%cd R3-RAG

# Télécharger un modèle plus petit
print("Téléchargement du modèle R3-RAG-CS-Qwen (7B)...")
!git lfs install

!git clone https://huggingface.co/Yuan-Li-FNLP/R3-RAG-CS-Qwen

In [ ]:
MODEL_PATH = "/content/R3-RAG/R3-RAG-CS-Qwen"

# Vérifier que le modèle existe
import os
if os.path.exists(MODEL_PATH):
    print(f"✅ Modèle trouvé : {MODEL_PATH}")
    print(f"Taille : {sum(os.path.getsize(os.path.join(MODEL_PATH, f)) for f in os.listdir(MODEL_PATH) if os.path.isfile(os.path.join(MODEL_PATH, f))) / (1024**3):.2f} GB")
else:
    print(f"❌ Modèle non trouvé : {MODEL_PATH}")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Charger le modèle avec précision réduite
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Modèle chargé avec succès !")
print(f"Modèle sur device : {model.device}")

In [ ]:
def test_model(question):
    prompt = f"""Question: {question}

Let me think step by step and retrieve relevant information.

Action: """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):]

In [ ]:
def run_comprehensive_test(model, tokenizer):
    """
    Test complet du modèle R3-RAG-CS
    """
    test_cases_cold_start = [
    {
        "category": "Factual - Very Easy",
        "question": "What is the capital of France?",
        "expected_type": "Direct fact"
    },
    {
        "category": "Factual - Easy",
        "question": "Who invented the light bulb?",
        "expected_type": "Single fact retrieval"
    },
    {
        "category": "Simple Multi-hop",
        "question": "What ocean is located east of the United States?",
        "expected_type": "Geographic reasoning"
    }
]

    results = []
    for test in test_cases_cold_start:
        print(f"\n{'='*60}")
        print(f"Category: {test['category']}")
        print(f"Question: {test['question']}")
        print(f"Expected: {test['expected_type']}")
        print(f"{'-'*60}")

        response = test_model(test['question'])
        print(f"Response: {response}")

        results.append({
            "question": test['question'],
            "response": response,
            "category": test['category']
        })

    return results

# Exécuter les tests
results = run_comprehensive_test(model, tokenizer)

## Test R3-RAG-Qwen (full)

In [ ]:
import os
from google.colab import drive

# Monter Google Drive (optionnel, pour sauvegarder)
drive.mount('/content/drive')

# Cloner le repository
!git clone https://github.com/Yuan-Li-FNLP/R3-RAG.git
%cd R3-RAG

# Télécharger un modèle plus petit (choisir selon la RAM disponible)
print("Téléchargement du modèle R3-RAG-CS-Qwen (7B)...")
!git lfs install

!git clone https://huggingface.co/Yuan-Li-FNLP/R3-RAG-Qwen

In [ ]:
MODEL_PATH = "/content/R3-RAG/R3-RAG-Qwen"

# Vérifier que le modèle existe
import os
if os.path.exists(MODEL_PATH):
    print(f"✅ Modèle trouvé : {MODEL_PATH}")
    print(f"Taille : {sum(os.path.getsize(os.path.join(MODEL_PATH, f)) for f in os.listdir(MODEL_PATH) if os.path.isfile(os.path.join(MODEL_PATH, f))) / (1024**3):.2f} GB")
else:
    print(f"❌ Modèle non trouvé : {MODEL_PATH}")

In [ ]:
MODEL_PATH = "/content/R3-RAG/R3-RAG-Qwen"

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Charger le tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

# Charger le modèle avec précision réduite
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Modèle chargé avec succès !")
print(f"Modèle sur device : {model.device}")

In [ ]:
def test_model(question):
    # Format de prompt pour R3-RAG
    prompt = f"""Question: {question}

Let me think step by step and retrieve relevant information.

Action: """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response[len(prompt):]

2min

In [ ]:
def run_comprehensive_test(model, tokenizer):
    """
    Test complet du modèle R3-RAG
    """
    test_cases = [
        {
            "category": "Factual - Easy",
            "question": "What is the capital of Japan?",
            "expected_type": "Single fact retrieval"
        }
    ]

    results = []
    for test in test_cases:
        print(f"\n{'='*60}")
        print(f"Category: {test['category']}")
        print(f"Question: {test['question']}")
        print(f"Expected: {test['expected_type']}")
        print(f"{'-'*60}")

        response = test_model(test['question'])
        print(f"Response: {response}")

        results.append({
            "question": test['question'],
            "response": response,
            "category": test['category']
        })

    return results

# Exécuter les tests
results = run_comprehensive_test(model, tokenizer)

In [ ]:
  def run_comprehensive_test(model, tokenizer):
    """
    Test complet du modèle R3-RAG
    """
    test_cases = [

        {
            "category": "Multi-hop - Hard",
            "question": "Who directed the movie that won the Academy Award for Best Picture in 2020?",
            "expected_type": "Complex retrieval + reasoning"
        }
    ]

    results = []
    for test in test_cases:
        print(f"\n{'='*60}")
        print(f"Category: {test['category']}")
        print(f"Question: {test['question']}")
        print(f"Expected: {test['expected_type']}")
        print(f"{'-'*60}")

        response = test_model(test['question'])
        print(f"Response: {response}")

        results.append({
            "question": test['question'],
            "response": response,
            "category": test['category']
        })

    return results

# Exécuter les tests
results = run_comprehensive_test(model, tokenizer)